# A degenerate diffusion equation with constant density

We want to solve the problem 
$$ \rho w - \nabla \cdot ( \rho ( u \otimes u) \nabla w) = f \text{ in } D,$$
where $D = [-1,1] \subset \mathbb{R}^2$. In this script, we only consider a constant density $\rho = 1$. 
We want to approximate the exact solution 
$$w := \exp (-6((x+0.5)^2 + y^2))  - \exp (-6((x-0.5)^2+y^2)).$$

First of all, we import the needed functionality from [NGSolve](https://ngsolve.org). 

In [8]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.meshes import *
from ngsolve.webgui import Draw

The following function calculates a right hand side for a given exact solution through
$$f = \rho w - \nabla \cdot (\rho (u \otimes u) \cdot \nabla w). $$

In [9]:
def calculate_rhs(u,exact,rho):
    umat = CoefficientFunction(u,dims=(2,1))
    uTen = umat*umat.trans
    exactGrad = (exact.Diff(x),exact.Diff(y))
    exactDiffusion = rho*uTen*exactGrad
    exactDiv = exactDiffusion[0].Diff(x)+exactDiffusion[1].Diff(y)
    rhs = rho*exact-exactDiv
    return rhs, exactGrad

Furthermore, we define a differential operator
$$ \partial_u \cdot := (u \cdot \nabla) \cdot . $$

In [10]:
du = lambda u,w: InnerProduct(u,grad(w))

Now, we implement a function solving discrete problem derived in the thesis:  \
Find $w_h \in V_h^{k,d}$ such that 
$$ \mathcal{B}_h (w_h,v_h) = F_h(v_h) \qquad \forall v_h \in V_h^{k,d},$$
where the bilinear form $\mathcal{B}_h(\cdot,\cdot)$ is defined through 
$$ \begin{align}
\mathcal{B}_h (w_h,v_h) = &\sum_{T \in \mathcal{T_h}} \int_T \rho w_h v_h dx + \int_T \rho \partial_u w_h \partial v_h dx \\ 
&+ \sum_{F \in \mathcal{F}_h} \int_F u_\nu \{ \! \! \{ - \rho \partial_u w_h \} \! \! \} [v_h] ds + \int_F u_\nu \{ \! \! \{ - \rho \partial_u v_h \} \! \! \} [w_h] ds + \int_F \frac{\rho \lambda}{h} |u_\nu|^2 [w_h] [v_h] ds,
\end{align}$$
and the linear form $ F_h (\cdot, \cdot)$ through
$$ F_h(v_h) = \int_D f v_h  dx + \sum_{F \in \mathcal{F}_h^{\partial D}} \int_F u_\nu \partial_u v_h w ds + \int_F \rho \frac{\lambda}{\rho} |u_\nu|^2v_h w ds.$$
Here we denote $u_\nu = u \cdot \nu$, where $\nu$ is the normal vector. Further, we use the average- and the jump operator defined through 
$$ \{ \! \! \{ w_h \} \! \! \} = \frac{1}{2}(w_h\vert_{T_1} + w_h\vert_{T_2}),$$
$$ [w_h] = w_h\vert_{T_1} - w_h\vert_{T_2} .$$

In [11]:
def Solve(k,exact,rho,uC,lamb,mesh):
    fes = L2(mesh, order=k, dgjumps=True)
    w,v = fes.TnT()
    gfu = GridFunction(fes)

    n = specialcf.normal(2)
    h = specialcf.mesh_size

    jump_w = w-w.Other()
    jump_v = v-v.Other()
    avg_duw = 0.5*(du(uC,w)+du(uC,w.Other()))
    avg_duv = 0.5*(du(uC,v)+du(uC,v.Other()))


    dX = dx(bonus_intorder = 2)
    dS = ds(bonus_intorder = 2)

    rhs,exactGrad = calculate_rhs(uC,exact,rho)

    a = BilinearForm(fes,symmetric=True)
    a += rho*w*v*dx
    a += rho*du(uC,w)*du(uC,v)*dX
    a += uC*n*-rho*avg_duw*jump_v*dX(skeleton=True)
    a += uC*n*-rho*avg_duv*jump_w*dX(skeleton=True)
    a += rho*lamb*1/h*(uC*n)*(uC*n)*jump_w*jump_v*dX(skeleton=True)
    a += uC*n*-rho*du(uC,w)*v*dS(skeleton=True)
    a += uC*n*-rho*du(uC,v)*w*dS(skeleton=True)
    a += rho*lamb*1/h*(uC*n)*(uC*n)*w*v*dS(skeleton=True)

    f = LinearForm(fes)
    f += rhs*v*dX
    f += uC*n*-rho*du(uC,v)*exact*dS(skeleton=True)
    f += rho*lamb*1/h*(uC*n)*(uC*n)*exact*v*dS(skeleton=True)

    a.Assemble()
    f.Assemble()

    aInv = a.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky")
    gfu.vec[:] = 0.0
    gfu.vec.data = aInv * f.vec

    return gfu, exactGrad

We implement the exact solution and three velocity fields to test the method: \
$$\begin{align}
u_0 &= (1,1), \\
u_1 &= (-0.75y,0.75x), \\
u_2 &= (2y(1-x^2),-2x(1-y^2)). \\
\end{align}$$

In [12]:
gaussp = CoefficientFunction(exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y)))
u0 = CoefficientFunction((1,1))
u1 = CoefficientFunction((-0.75*y,0.75*x))
u2 = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
uC = u2

Now, we define a structured mesh on the domain $D$.

In [13]:
n = 3
mesh = MakeStructured2DMesh(quads=False, nx=2**n, ny=2**n,mapping = lambda x,y: (2*x-1,2*y-1))

In [14]:
Draw(gaussp,mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2102-163-g1362a8d5f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

Now, we fix a polynomial degree $k$, a penalization parameter $\lambda$ and a constant density $\rho$.

In [15]:
k = 1
lamb = 10*(k+1)**2
rho = 1

Then, we can solve the solut

In [16]:
gfu, exactGrad = Solve(k,gaussp,rho,uC,lamb,mesh)
Draw(gfu,mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2102-163-g1362a8d5f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

We can calculate the error in the $L^2$- and the $W$-norm, where the latter is defined as 
$$ | \! | w | \! |_W^2 :=  | \! | \rho^{\frac{1}{2}} w | \! |_{L^2}^2 + | \! | \rho^{\frac{1}{2}} \partial_u w | \! |_{L^2}^2$$

In [17]:
L2error = sqrt(Integrate((gfu-gaussp)**2,mesh))
Werror = sqrt(Integrate(rho*(gfu-gaussp)**2,mesh)) + sqrt(Integrate(rho*(du(uC,gfu)-uC*exactGrad)**2,mesh))
print("L2-error: \n", L2error)
print("W-error: \n", Werror)

L2-error: 
 0.10091554514561535
W-error: 
 0.6710174439602203


With a simple loop, we can study the convergence of the method; for example in the $W$-norm: 

In [18]:
Werrors = []
for n in range(1,8):
    mesh = MakeStructured2DMesh(quads=False, nx=2**n, ny=2**n,mapping = lambda x,y: (2*x-1,2*y-1))
    gfu, exactGrad = Solve(k,gaussp,rho,u0,lamb,mesh)
    Werror = sqrt(Integrate(rho*(gfu-gaussp)**2,mesh)) + sqrt(Integrate(rho*(du(uC,gfu)-uC*exactGrad)**2,mesh))
    Werrors.append(Werror)
    print("Mesh generated with n=",n)
    if n > 1: 
        wrate = round(log(Werrors[n-2]/Werrors[n-1])/log(2),2)
        print("W-Error:",Werror)
        print("EOC:", wrate)
    else:
        print("W-Error:",Werror)

Mesh generated with n= 1
W-Error: 1.9863857050938158
Mesh generated with n= 2
W-Error: 1.2254870074593518
EOC: 0.7
Mesh generated with n= 3
W-Error: 0.906813639391026
EOC: 0.43
Mesh generated with n= 4
W-Error: 0.480897957510108
EOC: 0.92
Mesh generated with n= 5
W-Error: 0.22194062112804772
EOC: 1.12
Mesh generated with n= 6
W-Error: 0.09973727553999694
EOC: 1.15
Mesh generated with n= 7
W-Error: 0.04678265468593071
EOC: 1.09
